## Import libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from datetime import datetime, timedelta
import re



## Put credentials here

In [2]:
my_email = "axelberglund98@gmail.com"
my_password = "Password_1"


## Open Chrome Window

In [3]:
# Initialize the WebDriver
driver = webdriver.Chrome()
time.sleep(3) 


## Goto the website

In [4]:
driver.get("https://www.voyage-prive.co.uk/login/index#signin")
time.sleep(5)


## Press cookie button

In [5]:
#cookie_button=driver.find_element(By.CSS_SELECTOR, "#onetrust-close-btn-container > button")
cookie_button=driver.find_element(By.CSS_SELECTOR, "#cookieBanner > div > div.BannerContainer__buttons > button.Button.Button--light.Button--accept")
cookie_button.click()
time.sleep(2)


## Enter Username

In [6]:
email_field = driver.find_element(By.CSS_SELECTOR, "#reactLogin > div > div > div.Login__content > form > div > div > div:nth-child(1) > input[type=email]")
email_field.send_keys(my_email)
time.sleep(1)


## Enter Password

In [7]:
#password_field = driver.find_element(By.ID, "")
password_field = driver.find_element(By.CSS_SELECTOR, "#reactLogin > div > div > div.Login__content > form > div > div > div.Login__line.passwordContainer > div > input")
password_field.send_keys(my_password)
time.sleep(1)


## Press Login Button

In [8]:
login_button = driver.find_element(By.CSS_SELECTOR, "#reactLogin > div > div > div.Login__content > form > div > div > div.Login__line.Login__line--spaced > button")  # Replace with the actual ID or selector
login_button.click()
time.sleep(5)


## Now we are logged in

## Load all data by scrolling down

In [9]:

# Scroll down the page gradually to load all content
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load the content
        time.sleep(5)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_to_bottom(driver)

## TODO: Get the data

In [10]:
def add_dates_to_dataframe(df):
    """
    Add start_date and end_date columns to the DataFrame based on the current datetime
    and the 'time_remaining' column.

    Args:
        df (pd.DataFrame): Input DataFrame containing 'time_remaining' column.

    Returns:
        pd.DataFrame: DataFrame with additional 'start_date' and 'end_date' columns.
    """
    # Define the current datetime as the start date
    current_datetime = datetime.now()

    start_dates = []
    end_dates = []

    for time_remaining in df['time_remaining']:
        # Check for time in days
        match_days = re.search(r"(\d+)\s*days?", time_remaining, re.IGNORECASE)
        if match_days:
            days = int(match_days.group(1))
            time_delta = timedelta(days=days)
        else:
            # Parse the hours and minutes
            match_time = re.search(r"(\d+)h\s*(\d+)?m?", time_remaining, re.IGNORECASE)
            if match_time:
                hours = int(match_time.group(1))
                minutes = int(match_time.group(2)) if match_time.group(2) else 0
                time_delta = timedelta(hours=hours, minutes=minutes)
            else:
                time_delta = timedelta()

        end_datetime = current_datetime + time_delta

        # Append the start and end dates
        start_dates.append(current_datetime)
        end_dates.append(end_datetime)

    # Add the new columns to the DataFrame
    df['start_date'] = start_dates
    df['end_date'] = end_dates

    return df

## Remember to scroll down to get all the data

In [11]:
# Supprimer les publicités 
# English : Remove ads
ads = driver.find_elements(By.XPATH, "//article[contains(@id, 'push-promo')]")
for ad in ads:
    driver.execute_script("arguments[0].remove();", ad)

# Récupérer les hôtels après suppression des pubs
# English : Get hotels after removing ads
hotel_list = driver.find_element(By.CSS_SELECTOR,'#section-now> div')
hotel_elements = hotel_list.find_elements(By.TAG_NAME, "article")

hotel_sale_destination = []
hotel_name = []
hotel_sale_price = []
hotel_sale_time_remaining = []
exclusif = []

for hotel in hotel_elements:
    sale_content = None
    sale_price = None
    sale_time_remaining = None
    sale_destination = None
    name = None
    exclusif_status = 'no'
    
    sub_elements1 = hotel.find_elements(By.TAG_NAME, "a")
    if sub_elements1:
        sub_elements2 = sub_elements1[0].find_elements(By.TAG_NAME, "div")
        for element in sub_elements2:
            class_name = element.get_attribute("class")
            if class_name == 'sale_content':
                sale_content = element
            elif class_name == 'sale_price':
                sale_price = element.text
                if sale_price == 'EXCLUSIF':
                    exclusif_status = 'yes'

                    # Ouvrir le lien de l'hôtel dans un nouvel onglet pour récupérer le prix réel
                    link_element = hotel.find_element(By.TAG_NAME, "a")
                    link_url = link_element.get_attribute("href")
                    driver.execute_script("window.open(arguments[0], '_blank');", link_url)

                    # Passer à l'onglet nouvellement ouvert
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(2)  # Attendre le chargement de la page

                    try:
                        sale_price = driver.find_element(By.CSS_SELECTOR, ".offer-price.details-price.euro").text
                    except:
                        sale_price = "N/A"

                    # Fermer l'onglet et revenir à l'onglet principal
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            elif class_name == 'sale-time-remaining  specific-not-near ':
                sale_time_remaining = element.text
                
        if sale_content:
            sub_elements3 = sale_content.find_elements(By.TAG_NAME, "span")
            for element in sub_elements3:
                class_name = element.get_attribute("class")
                if class_name == 'sale-destination':
                    sale_destination = element.text
                elif class_name == 'hotel-name':
                    name = element.text

    
    hotel_sale_price.append(sale_price or "N/A")
    hotel_sale_time_remaining.append(sale_time_remaining or "N/A")
    hotel_sale_destination.append(sale_destination or "N/A")
    hotel_name.append(name or "N/A")
    exclusif.append(exclusif_status)


data = {
    "Country": hotel_sale_destination,
    "hotel_name": hotel_name,
    "price": hotel_sale_price,
    "time_remaining": hotel_sale_time_remaining,
}
df = pd.DataFrame(data)

df = add_dates_to_dataframe(df)


In [12]:
df.shape


(194, 6)

In [ ]:
# print todays date in a string
today = datetime.today().strftime('%d_%m_%Y')


17_01_2025


In [14]:

# Sauvegarder le DataFrame dans un fichier Excel
output_file = f"voyageprive_hotels_{today}.csv"


In [15]:
print(output_file)

voyageprive_hotels_17_01_2025.csv


In [16]:

df.to_csv(output_file, index=False)


In [17]:
df.head()

,Country,hotel_name,price,time_remaining,start_date,end_date
0,INDIAN OCEAN / MALDIVES,Cinnamon Velifushi Maldives 5*,From\n£850,10h 42m remaining,2025-01-10 14:17:25.833497,2025-01-11 00:59:25.833497
1,"TURKEY / ISTANBUL, CAPPADOCIA & PAMUKKALE",7-8 night tour: The wonders of Turkey,From\n£365,10h 42m remaining,2025-01-10 14:17:25.833497,2025-01-11 00:59:25.833497
2,ITALY / ISCHIA,7 night trip: Neapolitan Bay,From\n£567,10h 42m remaining,2025-01-10 14:17:25.833497,2025-01-11 00:59:25.833497
3,"INDIA / DELHI, JAIPUR & AGRA",7-11 night private tour: India and its Golden ...,From\n£536,10h 42m remaining,2025-01-10 14:17:25.833497,2025-01-11 00:59:25.833497
4,EGYPT / SHARM EL SHEIKH,Grand Rotana Resort & Spa 5*,Up to\n-75%,10h 42m remaining,2025-01-10 14:17:25.833497,2025-01-11 00:59:25.833497


## Quit

In [201]:
driver.quit()